Read an image file into an array

In [1]:
import numpy as np
from PIL import Image

def read_image_to_array(filepath):
    try:
        img=Image.open(filepath)
        #convert image to numpy array
        img_array=np.array(img)
        print(f"Image '{filepath}' read into array with shape: {img_array.shape}")
        return img_array
    except FileNotFoundError:
        print(f"Error: The file '{filepath} was not found")
        return None
def write_array_to_image(array,filepath):
    if array is None:
        print("Error:The input array is None")
        return
    try:
        #Ensure array is in unsigned 8-bit int and values clipped to 0-255
        clipped_array=np.clip(array,0,255).astype(np.uint8)

        #create an image from the array
        img=Image.fromarray(clipped_array)

        #save image to specified file
        img.save(filepath)
        print(f"Array Successfully written to '{filepath}'")
    except Exception as e:
        print(f"An error occured while writing the image: {e}")

color_image_array=read_image_to_array('/kaggle/input/dedeepya-images-ds/ds/My_image_color.jpeg')
if color_image_array is not None:
    write_array_to_image(color_image_array,'./0001_color.png')
grey_image_array=read_image_to_array('/kaggle/input/colour-greyscale-dataset/data/Flowers/flowers_grey/0005.png')
if grey_image_array is not None:
    write_array_to_image(grey_image_array,'./0005_grey.png')

Image '/kaggle/input/dedeepya-images-ds/ds/My_image_color.jpeg' read into array with shape: (960, 1280, 3)
Array Successfully written to './0001_color.png'
Image '/kaggle/input/colour-greyscale-dataset/data/Flowers/flowers_grey/0005.png' read into array with shape: (128, 128, 4)
Array Successfully written to './0005_grey.png'


In [2]:
def change_brightness(image_array,value):
    #Use floating point type for the calculation to prevent overflow 
    #then clip reslt back to valid range 0-255 fr 8-bit image
    bright_image_array=np.clip(image_array.astype(np.float32)+value,0,255)

    #convert back to original data type
    return bright_image_array.astype(bright_image_array.dtype)

original_image =read_image_to_array('/kaggle/input/dedeepya-images-ds/ds/My_image_color.jpeg')
brighter_image=change_brightness(original_image,50)#inc brightness
darker_image=change_brightness(original_image,-50)#dec brightness
write_array_to_image(brighter_image,'brighter.jpg')
write_array_to_image(darker_image,'darker.jpg')

Image '/kaggle/input/dedeepya-images-ds/ds/My_image_color.jpeg' read into array with shape: (960, 1280, 3)
Array Successfully written to 'brighter.jpg'
Array Successfully written to 'darker.jpg'


Contrast

In [3]:
def change_contrast(image_array, factor):
    # Convert to float for calculation
    img_float = image_array.astype(np.float32)
    
    # Apply the contrast formula
    contrast_image_array = factor * (img_float - 128) + 128
    
    # Clip the values to the 0-255 range
    contrast_image_array = np.clip(contrast_image_array, 0, 255)
    
    return contrast_image_array.astype(image_array.dtype)

# --- Example Usage ---
original_image = read_image_to_array('/kaggle/input/dedeepya-images-ds/ds/My_image_color_2.jpeg')
high_contrast = change_contrast(original_image, 1.5) # Increase contrast
low_contrast = change_contrast(original_image, 0.7)   # Decrease contrast
write_array_to_image(high_contrast, 'high_contrast.jpg')
write_array_to_image(low_contrast, 'low_contrast.jpg')

Image '/kaggle/input/dedeepya-images-ds/ds/My_image_color_2.jpeg' read into array with shape: (1280, 960, 3)
Array Successfully written to 'high_contrast.jpg'
Array Successfully written to 'low_contrast.jpg'


**Average Method**:  Gray= (R+G+B)/3

**Luminous Method** :
Common weights: 0.299𝑅+0.587𝐺+0.114𝐵 (from ITU-R BT.601 standard)

In [4]:
def color_to_grayscale(image_array, method='luminosity'):
    if image_array.ndim != 3 or image_array.shape[2] != 3:
        print("Error: Input must be a color image (3 channels).")
        return image_array
        
    R, G, B = image_array[:,:,0], image_array[:,:,1], image_array[:,:,2]
    
    if method == 'average':
        gray_array = (R.astype(float) + G.astype(float) + B.astype(float)) / 3
    elif method == 'luminosity':
        gray_array = 0.299 * R + 0.587 * G + 0.114 * B
    else:
        raise ValueError("Unknown method. Choose 'average' or 'luminosity'.")
        
    return gray_array.astype(image_array.dtype)

# --- Example Usage ---
color_image = read_image_to_array('/kaggle/input/dedeepya-images-ds/ds/pics (1).jpeg')
gray_luminosity = color_to_grayscale(color_image, method='luminosity')
gray_average = color_to_grayscale(color_image, method='average')
write_array_to_image(gray_luminosity, 'gray_luminosity.png')
write_array_to_image(gray_average, 'gray_average.png')

Image '/kaggle/input/dedeepya-images-ds/ds/pics (1).jpeg' read into array with shape: (575, 1280, 3)
Array Successfully written to 'gray_luminosity.png'
Array Successfully written to 'gray_average.png'


grey to pseudo-color

In [5]:
import numpy as np

def apply_pseudocolor(gray_array):
    if gray_array.ndim != 2:
        raise ValueError("Input must be a grayscale image (2D array).")
    
    # Ensure uint8
    gray_array = gray_array.astype(np.uint8)
    
    # Create LUT for "hot" colormap
    lut = np.zeros((256, 3), dtype=np.uint8)
    for i in range(256):
        if i < 85:
            lut[i] = [i * 3, 0, 0]              # Black → Red
        elif i < 170:
            lut[i] = [255, (i - 85) * 3, 0]     # Red → Yellow
        else:
            lut[i] = [255, 255, (i - 170) * 3]  # Yellow → White

    # Map grayscale values to colors using LUT
    color_image = lut[gray_array]
    
    return color_image

# --- Example usage ---
gray_image = read_image_to_array('/kaggle/working/0001_color.png')
if gray_image.ndim == 3:
    gray_image = color_to_grayscale(gray_image)

pseudo_color_image = apply_pseudocolor(gray_image)
write_array_to_image(pseudo_color_image, 'pseudo_color.png')

Image '/kaggle/working/0001_color.png' read into array with shape: (960, 1280, 3)
Array Successfully written to 'pseudo_color.png'


Green Screen Replacement

In [6]:
def replace_green_screen(foreground_img, background_img):
    
    # Ensure images are the same size
    if foreground_img.shape != background_img.shape:
        print("Error: Images must have the same dimensions.")
        # Resize background to match foreground as a fallback
        bg_pil = Image.fromarray(background_img)
        bg_pil = bg_pil.resize((foreground_img.shape[1], foreground_img.shape[0]))
        background_img = np.array(bg_pil)

    # Get R, G, B channels of the foreground
    R, G, B = foreground_img[:,:,0], foreground_img[:,:,1], foreground_img[:,:,2]
    
    # Define the green screen condition (this may need tuning)
    # A simple but effective condition: G is the dominant channel
    # and G is above a certain brightness threshold.
    green_mask = (G > 100) & (G > R * 1.2) & (G > B * 1.2)
    print(green_mask.shape)
    
    # Expand the mask from 2D to 3D to work with color images
    green_mask_3d = np.stack([green_mask] * 3, axis=-1)
    print(green_mask_3d.shape)
    
    # Use np.where: if mask is True, use background; otherwise, use foreground.
    composite_image = np.where(green_mask_3d, background_img, foreground_img)
    
    return composite_image.astype(np.uint8)

# --- Example Usage ---
foreground = read_image_to_array('/kaggle/input/green-background-ds/smiling-young-caucasian-woman-points.jpg')
background = read_image_to_array('/kaggle/input/dedeepya-images-ds/ds/pics (3).jpeg')
result = replace_green_screen(foreground, background)
write_array_to_image(result, 'composite_image.png')

Image '/kaggle/input/green-background-ds/smiling-young-caucasian-woman-points.jpg' read into array with shape: (5304, 7521, 3)
Image '/kaggle/input/dedeepya-images-ds/ds/pics (3).jpeg' read into array with shape: (575, 1280, 3)
Error: Images must have the same dimensions.
(5304, 7521)
(5304, 7521, 3)
Array Successfully written to 'composite_image.png'


Video Processing: Frames to Video and Transitions

In [7]:
import cv2

def video_to_frames(video_path):
    """Reads a video file and returns a list of its frames as NumPy arrays."""
    frames = []
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return []
        
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # OpenCV reads in BGR format, so convert to RGB for consistency with Pillow
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame_rgb)
        
    cap.release()
    print(f"Read {len(frames)} frames from {video_path}")
    return frames

def frames_to_video(frames, output_path, fps):
    """Writes a list of frames to a video file."""
    if not frames:
        print("Error: Frame list is empty.")
        return
        
    H, W, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4 file
    writer = cv2.VideoWriter(output_path, fourcc, fps, (W, H))
    
    for frame in frames:
        # Convert back to BGR for OpenCV writer
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        writer.write(frame_bgr)
        
    writer.release()
    print(f"Video saved to {output_path}")

def create_fade_transition(img1_path, img2_path, output_path, duration_sec=1, fps=30):
    """Creates a 1-second fade transition video from image1 to image2."""
    img1 = read_image_to_array(img1_path)
    img2 = read_image_to_array(img2_path)

    # Ensure images are the same size
    if img1.shape != img2.shape:
        print("Resizing image2 to match image1 for transition.")
        img2_pil = Image.fromarray(img2).resize((img1.shape[1], img1.shape[0]))
        img2 = np.array(img2_pil)

    num_frames = int(duration_sec * fps)
    transition_frames = []

    # Convert to float for accurate blending
    img1_float = img1.astype(np.float32)
    img2_float = img2.astype(np.float32)

    for i in range(num_frames):
        alpha = i / (num_frames - 1) # Alpha goes from 0 to 1
        
        # Alpha blending formula
        blended_frame = ((1 - alpha) * img1_float + alpha * img2_float)
        
        # Convert back to uint8 for saving
        transition_frames.append(blended_frame.astype(np.uint8))
        
    # Write the frames to a video
    frames_to_video(transition_frames, output_path, fps)

# --- Example Usage ---
create_fade_transition('/kaggle/input/dedeepya-images-ds/ds/pics (1).jpeg', '/kaggle/input/dedeepya-images-ds/ds/pics (3).jpeg', 'fade_transition.mp4')

Image '/kaggle/input/dedeepya-images-ds/ds/pics (1).jpeg' read into array with shape: (575, 1280, 3)
Image '/kaggle/input/dedeepya-images-ds/ds/pics (3).jpeg' read into array with shape: (575, 1280, 3)
Video saved to fade_transition.mp4
